In [1]:
import re
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data.sampler import WeightedRandomSampler
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from loguru import logger
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup, PreTrainedTokenizer
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Load Data

In [2]:
train_data_path = "/home/mdafifal.mamun/notebooks/triagerX/train_data.csv"
test_data_path = "/home/mdafifal.mamun/notebooks/triagerX/test_data.csv"

train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

In [3]:
train_df["raw_text"] = train_df.apply(
            lambda x: "Title: "
            + str(x["issue_title"])
            + "\nDescription: "
            + str(x["description"]),
            axis=1,
        )

test_df["raw_text"] = test_df.apply(
            lambda x: "Title: "
            + str(x["issue_title"])
            + "\nDescription: "
            + str(x["description"]),
            axis=1,
        )

### Developer Mapping

In [4]:
vm_users = ['pshipton', 'keithc-ca', 'gacholio', 'tajila', 'babsingh', 'JasonFengJ9', 'fengxue-IS', 'hangshao0', 'theresa.t.mammarella', 'ChengJin01', 'singh264', 'thallium', 'ThanHenderson']
jvmti_users = ['gacholio', 'tajila', 'babsingh', 'fengxue-IS']
jclextensions_users = ['JasonFengJ9', 'pshipton', 'keithc-ca']
test_users = ['LongyuZhang', 'annaibm', 'sophiaxu0424', 'KapilPowar', 'llxia']
build_users = ['adambrousseau', 'mahdipub']
gc_users = ['dmitripivkine', 'amicic', 'kangyining', 'LinHu2016']

# Putting them in dictionaries
components_developers_map = {
    'comp:vm': vm_users,
    'comp:jvmti': jvmti_users,
    'comp:jclextensions': jclextensions_users,
    'comp:test': test_users,
    'comp:build': build_users,
    'comp:gc': gc_users
}

expected_users = [user for user_list in components_developers_map.values() for user in user_list]

In [5]:
train_df = train_df[train_df["owner"].isin(expected_users)]
print("Total train issues after developer filtering:", len(train_df))

test_df = test_df[test_df["owner"].isin(expected_users)]
print("Total train issues after developer filtering:", len(test_df))

Total train issues after developer filtering: 1577
Total train issues after developer filtering: 398


In [6]:
developer_id_map = dict(zip(train_df["owner"], train_df["owner_id"]))
print(developer_id_map)

{'pshipton': 12, 'JasonFengJ9': 1, 'llxia': 11, 'keithc-ca': 10, 'hangshao0': 9, 'fengxue-IS': 7, 'babsingh': 5, 'LinHu2016': 2, 'thallium': 15, 'tajila': 14, 'amicic': 4, 'LongyuZhang': 3, 'gacholio': 8, 'dmitripivkine': 6, 'ChengJin01': 0, 'singh264': 13}


In [7]:
component_id_map = {'comp:build': 0, 'comp:gc': 1, 'comp:jclextensions': 2, 'comp:jvmti': 3, 'comp:test': 4, 'comp:vm': 5}


print(component_id_map)

{'comp:build': 0, 'comp:gc': 1, 'comp:jclextensions': 2, 'comp:jvmti': 3, 'comp:test': 4, 'comp:vm': 5}


In [8]:
train_df = train_df[train_df["component"].isin(component_id_map.keys())]
test_df = test_df[test_df["component"].isin(component_id_map.keys())]

In [9]:
train_df["component_id"] = train_df["component"].apply(lambda x: component_id_map[x])
test_df["component_id"] = test_df["component"].apply(lambda x: component_id_map[x])

# Load Models

In [10]:
from triagerx.model.lbt_p_deberta import LBTPDeberta
from triagerx.model.lbtp_bilstm import LBTPBiLSTM
from triagerx.loss.loss_functions import *

In [11]:
len(test_df.component_id.unique())

6

### Prepare Tokenizer

In [12]:
import yaml

In [13]:
with open("/home/mdafifal.mamun/notebooks/triagerX/triagerx/config/config.yaml", "r") as file:
    configs = yaml.safe_load(file)

In [14]:
tokenizer_config = configs["tokenizer"]

In [15]:
special_tokens = {
    "hex": "[HEX]",
    "timestamp": "[TIMESTAMP]",
    "float": "[FLOAT_VALUE]",
    "param": "[PARAM_VALUE]"
}

tokenizer = AutoTokenizer.from_pretrained(tokenizer_config["model_name"])
special_tokens_dict = {"additional_special_tokens": list(special_tokens.values())}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [16]:
def get_trained_model(weight_path: str, num_classes, unfrozen_layers, dropout, base_model, tokenizer):
    model = LBTPDeberta(
        output_size=num_classes, 
        unfrozen_layers=unfrozen_layers, 
        dropout=dropout, 
        base_model=base_model
    )

    model.base_model.resize_token_embeddings(len(tokenizer))

    model.load_state_dict(torch.load(weight_path))

    return model

In [17]:
component_model_weights = "/work/disa_lab/projects/triagerx/models/testing/comp_ex_LBTPDeberta_u5_6_classes_CombinedLoss.pt"
developer_model_weights = "/work/disa_lab/projects/triagerx/models/testing/dev_ex_LBTPDeberta_u4_cv9_classes16_CombinedLoss.pt"

In [18]:
len(test_df.component_id.unique())

6

In [19]:
component_model = get_trained_model(
    weight_path=component_model_weights,
    num_classes=len(test_df.component_id.unique()), 
    unfrozen_layers=5, 
    dropout=0.2, 
    base_model="microsoft/deberta-base",
    tokenizer=tokenizer
)


/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [20]:
developer_model = get_trained_model(
    weight_path=developer_model_weights,
    num_classes=len(train_df.owner_id.unique()), 
    unfrozen_layers=4, 
    dropout=0.3, 
    base_model="microsoft/deberta-large",
    tokenizer=tokenizer
)

In [21]:
similarity_model = SentenceTransformer('all-mpnet-base-v2')
# all_train_embeddings = similarity_model.encode(train_df.raw_text.to_list(), batch_size=15)

In [22]:
from triagerx.system.triagerx import TriagerX

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
trx = TriagerX(
    developer_prediction_model=developer_model,
    component_prediction_model=component_model,
    similarity_model=similarity_model,
    tokenizer=tokenizer,
    tokenizer_config=tokenizer_config,
    train_data=train_df,
    developer_id_map=developer_id_map,
    component_id_map=component_id_map,
    component_developers_map=components_developers_map,
    issues_path="/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/issue_data",
    device=device
)

Using device: cuda


2024-05-02 23:13:23.367 | DEBUG    | triagerx.system.triagerx:__init__:53 - Generating embedding for existing issues...


In [24]:
def get_recommendation_for_test_index(test_df, test_idx):
    actual_owner = test_df.iloc[test_idx].owner
    actual_component = test_df.iloc[test_idx].component
    issue_url = test_df.iloc[test_idx].issue_url

    print("Issue:", test_df.iloc[test_idx].issue_url)
    print("Actual component:", actual_component)
    print("Actual owner:", actual_owner)
    recommendation = trx.get_recommendation(
        test_df.iloc[test_idx].text,
        k_comp=3,
        k_dev=3,
        k_rank=5,
        similarity_threshold=0.5
    )

    status = None

    if actual_owner in recommendation["predicted_developers"]:
        status = "Matched"
    else:
        status = "Not Matched"

    recommendation["actual_owner"] = actual_owner
    recommendation["issue_url"] = issue_url
    recommendation["actual_component"] = actual_component
    recommendation["model_pred_status"] = status
    recommendation["rank_status"] = "Matched" if actual_owner in recommendation["combined_ranking"][:3] else "Not Matched"

    return recommendation

In [25]:
recommendations_matched = []
component_matched = []
ranked_matched = []

for idx in range(len(test_df)):
    rec = get_recommendation_for_test_index(test_df, idx)
    
    if rec["model_pred_status"] == "Matched":
        recommendations_matched.append(rec)
    
    if rec["actual_component"] in rec["predicted_components"]:
        component_matched.append(rec)

    if rec["rank_status"] == "Matched":
        ranked_matched.append(rec)

# rec_df = pd.DataFrame(recommendations_matched)
# rec_df.to_csv("recommendations_matched.csv")

# rec_df = pd.DataFrame(recommendations_unmatched)
# rec_df.to_csv("recommendations_unmatched.csv")

2024-05-02 23:13:31.708 | DEBUG    | triagerx.system.triagerx:get_recommendation:57 - Processing issue...
2024-05-02 23:13:31.712 | DEBUG    | triagerx.system.triagerx:get_recommendation:61 - Prediciting components...


Issue: https://github.com/eclipse-openj9/openj9/issues/5796
Actual component: comp:vm
Actual owner: pshipton


2024-05-02 23:13:31.908 | INFO     | triagerx.system.triagerx:get_recommendation:64 - Predicted components: ['comp:vm', 'comp:test', 'comp:jvmti']
2024-05-02 23:13:31.911 | DEBUG    | triagerx.system.triagerx:get_recommendation:66 - Generating developer recommendation...
2024-05-02 23:13:31.971 | INFO     | triagerx.system.triagerx:get_recommendation:69 - Recommended developers: ['JasonFengJ9', 'pshipton', 'hangshao0', 'babsingh', 'tajila']
2024-05-02 23:13:31.972 | DEBUG    | triagerx.system.triagerx:get_recommendation:71 - Generating recommendation by similarity...
2024-05-02 23:13:31.996 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:180 - Skipping issue as label id 2 did not match with any of [5, 4, 3]
2024-05-02 23:13:31.998 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:198 - Skipped users: ['ajohnson1', 'ajohnson1', 'paulcheeseman', 'paulcheeseman', 'paulcheeseman', 'paulcheeseman', 'ajohnson1'] because they don't exist in the ${expected_d

Issue: https://github.com/eclipse-openj9/openj9/issues/17869
Actual component: comp:vm
Actual owner: babsingh
Issue: https://github.com/eclipse-openj9/openj9/issues/11116
Actual component: comp:build
Actual owner: babsingh


2024-05-02 23:13:32.231 | INFO     | triagerx.system.triagerx:get_recommendation:69 - Recommended developers: ['keithc-ca', 'babsingh', 'JasonFengJ9', 'tajila', 'pshipton']
2024-05-02 23:13:32.232 | DEBUG    | triagerx.system.triagerx:get_recommendation:71 - Generating recommendation by similarity...
2024-05-02 23:13:32.253 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:198 - Skipped users: ['r30shah', 'r30shah'] because they don't exist in the ${expected_developers} list
2024-05-02 23:13:32.255 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:164 - [('JasonFengJ9', 13.697406351566315), ('ChengJin01', 8.472741544246674), ('babsingh', 5.730522871017456), ('pshipton', 4.5692238211631775), ('fengxue-IS', 4.359413534402847), ('keithc-ca', 3.64415243268013), ('ThanHenderson', 3.5303089767694473)]
2024-05-02 23:13:32.256 | INFO     | triagerx.system.triagerx:get_recommendation:80 - Recommended developers by issue similarity: ['JasonFengJ9', 'ChengJi

...

2024-05-02 23:14:07.336 | INFO     | triagerx.system.triagerx:get_recommendation:69 - Recommended developers: ['gacholio', 'fengxue-IS', 'babsingh', 'hangshao0', 'pshipton']
2024-05-02 23:14:07.338 | DEBUG    | triagerx.system.triagerx:get_recommendation:71 - Generating recommendation by similarity...
2024-05-02 23:14:07.350 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:164 - [('gacholio', 0.6315175443887711)]
2024-05-02 23:14:07.351 | INFO     | triagerx.system.triagerx:get_recommendation:80 - Recommended developers by issue similarity: ['gacholio']
2024-05-02 23:14:07.353 | DEBUG    | triagerx.system.triagerx:get_recommendation:82 - Aggregating ranking...
2024-05-02 23:14:07.354 | INFO     | triagerx.system.triagerx:get_recommendation:89 - Recommended developers by ranking aggregation: ['gacholio', 'fengxue-IS', 'babsingh']
2024-05-02 23:14:07.356 | DEBUG    | triagerx.system.triagerx:get_recommendation:57 - Processing issue...
2024-05-02 23:14:07.360 | DEBU

Issue: https://github.com/eclipse-openj9/openj9/issues/14495
Actual component: comp:test
Actual owner: llxia
Issue: https://github.com/eclipse-openj9/openj9/issues/8514
Actual component: comp:vm
Actual owner: babsingh


2024-05-02 23:14:07.575 | INFO     | triagerx.system.triagerx:get_recommendation:69 - Recommended developers: ['babsingh', 'keithc-ca', 'ChengJin01', 'hangshao0', 'gacholio']
2024-05-02 23:14:07.576 | DEBUG    | triagerx.system.triagerx:get_recommendation:71 - Generating recommendation by similarity...
2024-05-02 23:14:07.600 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:198 - Skipped users: ['Hello71', 'Hello71', 'Hello71', 'Hello71', 'cedrichansen', 'cedrichansen', 'Hello71', 'Hello71', 'dnakamura', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'cedrichansen'] because they don't exist in the ${expected_developers} list
2024-05-02 23:14:07.601 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:164 - [('gacholio', 31.707414537668228), ('babsingh', 8.5605338960886), ('pshipton', 7.470574826002121), ('keithc-ca', 6.99490562081337)]
2024-05-02 23:14:07.603 | INFO     | triagerx.system.triagerx:get_recommen

Issue: https://github.com/eclipse-openj9/openj9/issues/13848
Actual component: comp:vm
Actual owner: hangshao0
Issue: https://github.com/eclipse-openj9/openj9/issues/7349
Actual component: comp:vm
Actual owner: keithc-ca


2024-05-02 23:14:07.838 | INFO     | triagerx.system.triagerx:get_recommendation:69 - Recommended developers: ['babsingh', 'hangshao0', 'gacholio', 'ChengJin01', 'tajila']
2024-05-02 23:14:07.840 | DEBUG    | triagerx.system.triagerx:get_recommendation:71 - Generating recommendation by similarity...
2024-05-02 23:14:07.865 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:198 - Skipped users: ['manqingl', 'manqingl', 'manqingl', 'manqingl', 'AlenBadel', 'AlenBadel', 'AlenBadel'] because they don't exist in the ${expected_developers} list
2024-05-02 23:14:07.866 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:164 - [('keithc-ca', 10.225741192698479), ('hangshao0', 6.606559157371521), ('tajila', 3.2452626526355743), ('gacholio', 1.7246094346046448), ('pshipton', 1.6987474262714386), ('fengxue-IS', 1.6607587039470673)]
2024-05-02 23:14:07.868 | INFO     | triagerx.system.triagerx:get_recommendation:80 - Recommended developers by issue similarity: [

In [26]:
dev_model_score = len(recommendations_matched) / len(test_df)
ranked_score = len(ranked_matched) / len(test_df)
component_model_score = len(component_matched) / len(test_df)

print("Top 3 Recommendation Score")
print("=============================================")
print(
    f"Developer Recommendation Model: {dev_model_score}\n"
    f"Ranking Aggregation Score: {ranked_score}\n"
    f"Component Prediction Score: {component_model_score}\n"
)

Top 3 Recommendation Score
Developer Recommendation Model: 0.5958188153310104
Ranking Aggregation Score: 0.7003484320557491
Component Prediction Score: 0.9825783972125436



In [42]:
from pprint import pprint

pprint(ranked_matched[29])

{'actual_component': 'comp:vm',
 'actual_owner': 'keithc-ca',
 'combined_ranking': ['hangshao0', 'pshipton', 'keithc-ca'],
 'comp_prediction_score': [18.325912475585938,
                           -0.08967626094818115,
                           -5.03890323638916],
 'dev_prediction_score': [5.253453254699707,
                          4.881331443786621,
                          4.414284706115723,
                          4.388887405395508,
                          3.6985011100769043],
 'issue_url': 'https://github.com/eclipse-openj9/openj9/issues/1769',
 'model_pred_status': 'Not Matched',
 'predicted_components': ['comp:vm', 'comp:test', 'comp:build'],
 'predicted_developers': ['hangshao0', 'pshipton', 'JasonFengJ9'],
 'rank_status': 'Matched',
 'similar_devs': [('keithc-ca', 144.03236255049706),
                  ('pshipton', 40.27608476579189),
                  ('hangshao0', 12.830808162689209),
                  ('JasonFengJ9', 11.150637716054916)]}
